# Hello Observable World!

https://rxpy.readthedocs.io/en/latest/get_started.html

In [7]:
import reactivex
from reactivex import of, create, operators as ops

In [9]:
def my_first_observable(observer, scheduler):
    observer.on_next("Alpha")
    observer.on_next("Beta")
    observer.on_next("Gamma")
    observer.on_next("Delta")
    observer.on_next("Epsilon")
    observer.on_completed()
    observer.on_error(Exception("ma bad"))

source = create(my_first_observable)

source.subscribe(
    on_next=lambda value: print("Received {0}".format(value)),
    on_error=lambda error: print("Error Occurred: {0}".format(error)),
    on_completed=lambda: print("Done!"),
)

Received Alpha
Received Beta
Received Gamma
Received Delta
Received Epsilon
Done!


## OK now it's time to bring in some market data
### _ccxt_ is a sweet package for doing just that

In [5]:
import ccxt
from reactivex import of, operators as op

exchange = ccxt.kucoin()
exchange.load_markets()

COINS_TO_WATCH = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "XRP/USDT", "LTC/USDT"]

def refresh_tickers():
    source = of(*COINS_TO_WATCH)
    composed = source.pipe(
        op.map(lambda s: exchange.fetch_ticker(s)),
    )
    pairs = {}

    def add_to_pairs(value):
        pairs[value['symbol']] = value
        print(f"Received Value {value}!!")
        
    composed.subscribe(
        on_next=lambda value: add_to_pairs(value),
        on_error=lambda error: print(f"Error Occurred: {error}"),
        on_completed=lambda: print("Done!"),
    )
    return pairs

### Sweet! So now we have to work each of those JSON bodies into a prompt for GPT to give us a breakdown of what's going on

In [31]:
import os
from dotenv import load_dotenv
import json
import yaml
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


load_dotenv()

oai_org = "org-VAzYa2yfTTttUMNvCBhluUKn"
oai_key = os.getenv("OPENAI_API_KEY")

gpt = OpenAI(temperature=0.9)
market_prompt = ("give a high-level human summary about the trajectory of the"
    + " trading pair, as well as any more info you need to suggest a "
    + "market strategy:\n::::\n {market_data}")
market_data = pairs["ADA/USDT"]
market_prompt += json.dumps(market_data)
print(gpt(market_prompt))
# prompt = PromptTemplate(
#     input_variables=['product'],
#     template="WHat is a good name for a company that makes {}?"
# )
# foo = prompt.format(product="socks")
# print(foo)
# gpt = OpenAI(openai_api_key=oai_key, openai_organization=oai_org, temperature=0.9)
# text = "What would be a good company name for a company that makes colorful socks?"
# print(gpt(text))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




The trading pair for ADA/USDT has seen some recent volatility, as the high for the pair was 0.3675 and the low was 0.359903. The last price for the pair was 0.364827, which is a 0.43% decrease from the open price of 0.36641. Based on the current market conditions, this could indicate a bearish trajectory for the pair. Market strategy should take into consideration the current volatility and focus on bearish strategies in order to maximize potential profits.


## THis is awesome, now let's try to apply the _PromptTemplate_ Module to organize all these prompts

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

gpt = OpenAI(temperature=0.75)
prompt_txt = (
    "give a high-level human summary about the trajectory of the" +
    " trading pair, as well as any more info you need to suggest a " +
    "market strategy:\n::::\n {market_data}")

lvl_1_market_prompt = PromptTemplate(
    input_variables=['market_data'],
    template=prompt_txt
)

chain = LLMChain(llm = gpt, prompt=lvl_1_market_prompt)


__And now the secret sauce: composing all the tickers together!__

In [7]:
fresh_data = refresh_tickers()
token_analysis = {}
for k, v in fresh_data.items():
    analysis = chain.run(market_data=v)
    token_analysis[k] = analysis
    print(f"{k}: {analysis}")
# chain.run(market_data=pairs["ADA/USDT"])

Received Value {'symbol': 'BTC/USDT', 'timestamp': 1683690782219, 'datetime': '2023-05-10T03:53:02.219Z', 'high': 27844.0, 'low': 27353.0, 'bid': 27684.2, 'bidVolume': None, 'ask': 27684.3, 'askVolume': None, 'vwap': 27600.37972949078, 'open': 27599.3, 'close': 27684.2, 'last': 27684.2, 'previousClose': None, 'change': 84.9, 'percentage': 0.3, 'average': 27571.00118449, 'baseVolume': 2803.3630943, 'quoteVolume': 77373885.92232028, 'info': {'time': 1683690782219, 'symbol': 'BTC-USDT', 'buy': '27684.2', 'sell': '27684.3', 'changeRate': '0.003', 'changePrice': '84.9', 'high': '27844', 'low': '27353', 'vol': '2803.3630943', 'volValue': '77373885.922320277', 'last': '27684.2', 'averagePrice': '27571.00118449', 'takerFeeRate': '0.001', 'makerFeeRate': '0.001', 'takerCoefficient': '1', 'makerCoefficient': '1'}}!!
Received Value {'symbol': 'ETH/USDT', 'timestamp': 1683690782299, 'datetime': '2023-05-10T03:53:02.299Z', 'high': 1861.9, 'low': 1831.2, 'bid': 1843.9, 'bidVolume': None, 'ask': 1843

In [16]:
import json
print(json.dumps(token_analysis, indent=2))

{
  "BTC/USDT": "\n\nThe trading pair BTC/USDT has seen an increase of 0.3% over the past day, with the price rising from 27599.3 to 27684.2. The highest price of the day was 27844.0 and the lowest was 27353.0. The total volume of trading was 2803.3630943 and the total value of trading was 77373885.92232028. Based on this information, it is likely that BTC/USDT is in an upward trend and traders should look for buying opportunities.",
  "ETH/USDT": "\n\nThe trading pair ETH/USDT has been steadily rising in recent weeks, with its high of 1861.9, low of 1831.2, and close of 1844.07. There has been an increase of 0.59 in the price and 0.03 in the percentage. The volume of trading has also been high, with a base volume of 16918.7582843 and a quote volume of 31214440.267887823. Based on this information, it appears that the market for ETH/USDT is bullish and could be a good opportunity for traders. As such, a market strategy that takes into account the current market conditions and the poten

**THis is amazing... now let's begin to teach it about market-making (scripts)**

parameter_dict = {
    
}